In this script we dispose of original object ID's and remove duplicate vectors.

In [3]:
cfg={
    'main_folder':'/Users/matejkvassay/data/sketch-testing/',
    'dataset_file':'profi-neuralnet-100K.data'
}

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split

In [16]:
dataset_path=cfg['main_folder']+cfg['dataset_file']
dataset_new_path=cfg['main_folder']+'dataset.npy'
testset_path=cfg['main_folder']+'testset.npy'

In [6]:
class CaffeVectorsIterator(object):
    def __init__(self, file_path, limit=None):
        self.f = open(file_path, 'rb')
        self.limit=limit
        if self.limit is not None:
            self.returned=0

    def next(self):
        try:
            # (id: int, vector: np array of float32)
            if self.limit is not None:
                if self.returned==self.limit:
                    self.f.close()
                    raise StopIteration
                    
            lineA = self.f.next()
            lineB = self.f.next()
            if self.limit is not None:
                self.returned+=1
                
            return (int(lineA.split(' ')[2]),  np.fromstring(lineB, dtype='f', sep=' '))
        except StopIteration:
            self.f.close()
            raise StopIteration

    def __del__(self):
        if self.f:
            self.f.close()


class CaffeVectorsIterable(object):
    def __init__(self, file_path, limit=None):
        self.file_path = file_path
        self.limit=limit

    def __iter__(self):
        return CaffeVectorsIterator(self.file_path,limit=self.limit)

In [7]:
_, decaf_vecs=zip(*[obj for obj in CaffeVectorsIterable(dataset_path)])

In [8]:
decaf_vecs=np.matrix(decaf_vecs)

In [9]:
decaf_vecs.shape

(100000, 4096)

Remove duplicates

In [10]:
vecs=set()
indicies=list()
for i,vec in enumerate(decaf_vecs):
    key=str(vec)
    if not key in vecs:
        indicies.append(i)
        vecs.add(key)


In [11]:
decaf_vecs=decaf_vecs[np.array(indicies)]

Shuffle

In [13]:
np.random.shuffle(decaf_vecs)

Split

In [17]:
decaf_vecs, test_decaf_vecs=train_test_split(decaf_vecs,test_size=0.2)

In [20]:
decaf_vecs.shape

(69318, 4096)

In [21]:
test_decaf_vecs.shape

(17330, 4096)

Persist

In [18]:
with open(dataset_new_path, 'wb') as f:
    np.save(f,decaf_vecs)

In [19]:
with open(testset_path, 'wb') as f:
    np.save(f,test_decaf_vecs)